<a href="https://www.kaggle.com/code/aleksandrmorozov123/image-manipulations?scriptVersionId=230923259" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

**Deep fakes generations with PyTorch**

In [4]:
import os
if not os.path.exists('Faceswap-Deepfake-Pytorch'):
    !wget -q https://www.dropbox.com/s/5ji7jl7httso9ny/person_images.zip
    !wget -q https://raw.githubusercontent.com/sizhky/deep-fake-util/main/random_warp.py
    !unzip -q person_images.zip

!pip install -q torch_snippets torch_summary

from torch_snippets import *
from random_warp import get_training_data
import cv2

[person_images.zip]
  End-of-central-directory signature not found.  Either this file is not
  a zipfile, or it constitutes one disk of a multi-part archive.  In the
  latter case the central directory and zipfile comment will be found on
  the last disk(s) of this archive.
unzip:  cannot find zipfile directory in one of person_images.zip or
        person_images.zip.zip, and cannot find person_images.zip.ZIP, period.


In [5]:
face_cascade = cv2.CascadeClassifier (cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

In [6]:
def crop_face (img):
    gray = cv2.cvtColor (img, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale (gray, 1.3, 5)
    if (len(faces)>0):
        for (x, y, w, h) in faces:
            img2 = img[y: (y + h), x: (x + w), :]
        img2 = cv2.resize (img2, (256, 256))
        return img2, True
    else:
        return img, False

In [7]:
!mkdir cropped_faces_personA
!mkdir cropped_faces_personB

def crop_images (folder):
    images = Glob (folder + '/*.jpg')
    for i in range (len (images)):
        img = read (images[i], 1)
        img2, face_detected = crop_face (img)
        if (face_detected == False):
            continue
        else:
            cv2.imwrite ('cropped_faces_' + folder + '/' + str(i) + '.jpg', cv2.cvtColor(img2, cv2.COLOR_RGB2BGR))
crop_images ('personA')
crop_images ('personB')

In [8]:
class ImageDataset (Dataset):
    def __init__ (self, items_A, items_B):
        self.items_A = np.concatenate ([read (f, 1)[None] for f in items_A])/255.
        self.items_B = np.concatenate ([read (f, 1)[None] for f in items_B])/255.
        self.items_A += self.items_B.mean (axis = (0, 1, 2)) - self.items_A.mean (axis = (0, 1, 2))

    def __len__ (self):
        return min (len (self.items_A), len (self.items_B))
    def __getitem__ (self, ix):
        a, b = choose (self.items_A), choose (self.items_B)
        return a, b

    def collate_fn (self, batch):
        imsA, imsB = list (zip(*batch))
        imsA, targetA = get_training_data (imsA, len (imsA))
        imsB, targetB = get_training_data (imsB, len (imsB))
        imsA, imsB, targetA, targetB = [torch.Tensor(i).permute (0, 3, 1, 2).to(device) for i in [imsA, imsB, targetA, targetB]]

a = ImageDataset (Glob ('cropped_faces_personA'), Glob ('cropped_faces_personB'))
x = DataLoader (a, batch_size = 32, collate_fn = a.collate_fn)

NameError: name 'Dataset' is not defined